## import


In [ ]:
!pip install category_encoders
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
plt.rc('font', family='NanumBarunGothic') 
mpl.rc('axes', unicode_minus=False)
plt.rcParams["font.family"] = 'nanummyeongjo'
plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (12,6)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["axes.formatter.limits"] = -10000, 10000
%matplotlib inline
%config InlineBackend.figure_format='retina'

# 1. Preparation

## 1.1 Load datasets

In [7]:
data = pd.read_csv('/content/drive/MyDrive/final_data (1).csv')
df = data.copy()
df = df.drop(columns=['Unnamed: 0','거래일'])
num_feature = ['세대','인구','자동차등록','인구이동(전입지별)',
               '출생아수(명)', '사망자수(명)' ,'혼인건수(건)','이혼건수(건)']
for col in num_feature:
  df[col] = df[col].str.replace(",","").astype(int)
df['평당가'] = df['거래금액'] / df['전용면적']
df['거래횟수'] = [1]*len(df.index)

In [8]:
from sklearn.cluster import KMeans
구별랭킹 = pd.concat([df.groupby('지역코드').mean()[['평당가','거래금액']].sort_values('평당가',ascending=False),                 
                      df.groupby('지역코드').sum()[['거래횟수']]],
                      axis = 1)
kmeans = KMeans(n_clusters=5)
kmeans.fit(구별랭킹)
구별랭킹['구별K'] = kmeans.labels_
구별랭킹

,평당가,거래금액,거래횟수,구별K
강남구,1965.900547,172790.735915,12265,0
서초구,1726.341862,163525.710964,9850,0
송파구,1423.073380,117229.070079,15240,3
용산구,1417.798535,145481.003948,4812,0
성동구,1266.980612,98494.772688,9344,4
마포구,1191.961147,90848.400701,8842,4
광진구,1114.909593,93132.966703,4535,4
동작구,1071.001067,85849.420518,8461,4
영등포구,1029.975473,78392.765241,10121,1
강동구,1027.079886,75117.814662,12399,1


In [9]:
gu_mapper = {gu:k for gu,k in zip(구별랭킹.index,구별랭킹.구별K)}
df['구별K'] = df['지역코드'].copy()
df['구별K'] = df['구별K'].replace(gu_mapper)

## 1.2 Prepare datasets

In [10]:
dataset = df.drop(columns=['거래횟수','평당가','지번','-\xa0집세'])
dataset.columns = ['지역코드', '법정동', '아파트', '전용면적', '층', '건축년도', '거래금액', '거래년도', '거래월일', '세대',
       '인구', '세대당인구', '자동차등록', '인구이동(전입지별)', '외국인증권투자', '국고채 3년(평균)',
       '국고채 5년(평균)', '국고채 10년(평균)', '회사채 3년(평균)', 'CD 91물(평균)', '콜금리(1일물,평균)',
       '소비자물가', '- 농축수산물', '- 공업제품', '- 공공서비스', '- 개인서비스', '근원물가', '출생아수(명)',
       '사망자수(명)', '혼인건수(건)', '이혼건수(건)', 'KS11', 'KQ11', 'DJI', 'IXIC', 'VIX',
       'CSI300', 'SSEC', 'DE30', 'FCHI', 'NG', 'GC', 'HG', 'CL', '구별K']

dataset = dataset.drop(columns=['세대당인구','국고채 3년(평균)','국고채 5년(평균)','- 개인서비스','콜금리(1일물,평균)','CSI300','HG'])

In [11]:
def engine(xxx):
  xxx['KRX'] = xxx.KS11 + xxx.KQ11
  xxx = xxx.drop(columns=['KS11','KQ11'])
  return xxx

DF_1 = engine(dataset.copy())
DF_1.shape

(244456, 37)

In [12]:
dataset = DF_1

## 1.3 Train, Val, Test split

In [13]:
target='거래금액'
train = dataset[(dataset.거래월일 != '2021-01') | (dataset.거래월일 != '2021-02')].copy()
test = dataset[(dataset.거래월일 == '2021-01') | (dataset.거래월일 == '2021-02')].copy()
print(train.shape,test.shape)
print(test.거래월일.value_counts())

(244456, 37) (5248, 37)
2021-01    4826
2021-02     422
Name: 거래월일, dtype: int64


In [14]:
X = train.drop(columns=target)
y = train[target]
X_test = test.drop(columns=target)
y_test = test[target]
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

# 2. Baseline

In [17]:
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder

## 2.1 Linear

In [13]:
from sklearn.linear_model import LinearRegression
pipe = make_pipeline(
    OrdinalEncoder(), 
    LinearRegression()
)

pipe.fit(X_train, y_train)
print('훈련 R^2: ', pipe.score(X_train, y_train))
print('검증 R^2: ', pipe.score(X_val, y_val))
print('TEST R^2: ', pipe.score(X_test, y_test))

print('\n훈련 MAE: ', mean_absolute_error(pipe.predict(X_train), y_train))
print('검증 MAE: ', mean_absolute_error(pipe.predict(X_val), y_val))
print('TEST MAE: ', mean_absolute_error(pipe.predict(X_test), y_test))

훈련 R^2:  0.43149993942584997
검증 R^2:  0.4349519916586425
TEST R^2:  0.4540935651652793

훈련 MAE:  27874.76841577561
검증 MAE:  27972.489567581953
TEST MAE:  32607.446603240183


## 2.1 RandomForest

In [14]:
from sklearn.ensemble import RandomForestRegressor
pipe = make_pipeline(
    OrdinalEncoder(), 
    RandomForestRegressor(n_jobs=-1)
)

pipe.fit(X_train, y_train)
print('훈련 R^2: ', pipe.score(X_train, y_train))
print('검증 R^2: ', pipe.score(X_val, y_val))
print('TEST R^2: ', pipe.score(X_test, y_test))

print('\n훈련 MAE: ', mean_absolute_error(pipe.predict(X_train), y_train))
print('검증 MAE: ', mean_absolute_error(pipe.predict(X_val), y_val))
print('TEST MAE: ', mean_absolute_error(pipe.predict(X_test), y_test))

훈련 R^2:  0.9960248359625963
검증 R^2:  0.9706950473155046
TEST R^2:  0.9864338643789887

훈련 MAE:  1811.6328141142526
검증 MAE:  4822.040167983378
TEST MAE:  3836.8820698641057


## 2.2 XGBoost

In [16]:
from xgboost.sklearn import XGBModel
from xgboost import XGBRegressor
pipe = make_pipeline(
    OrdinalEncoder(), 
    XGBRegressor(n_jobs=-1)
)

pipe.fit(X_train, y_train)
print('훈련 R^2: ', pipe.score(X_train, y_train))
print('검증 R^2: ', pipe.score(X_val, y_val))
print('TEST R^2: ', pipe.score(X_test, y_test))

print('\n훈련 MAE: ', mean_absolute_error(pipe.predict(X_train), y_train))
print('검증 MAE: ', mean_absolute_error(pipe.predict(X_val), y_val))
print('TEST MAE: ', mean_absolute_error(pipe.predict(X_test), y_test))

[08:28:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
훈련 R^2:  0.8503150891645357
검증 R^2:  0.8468487199042718
TEST R^2:  0.8168025844056047

훈련 MAE:  14304.449106635033
검증 MAE:  14447.02974433746
TEST MAE:  18801.782822492645


## 2.3 LGBM

In [17]:
from lightgbm import LGBMRegressor
pipe = make_pipeline(
    OrdinalEncoder(), 
    LGBMRegressor(n_jobs=-1)
)

pipe.fit(X_train, y_train)
print('훈련 R^2: ', pipe.score(X_train, y_train))
print('검증 R^2: ', pipe.score(X_val, y_val))
print('TEST R^2: ', pipe.score(X_test, y_test))

print('\n훈련 MAE: ', mean_absolute_error(pipe.predict(X_train), y_train))
print('검증 MAE: ', mean_absolute_error(pipe.predict(X_val), y_val))
print('TEST MAE: ', mean_absolute_error(pipe.predict(X_test), y_test))

훈련 R^2:  0.9327245678096869
검증 R^2:  0.9275540035903953
TEST R^2:  0.9153730057689193

훈련 MAE:  9757.644952751476
검증 MAE:  9942.288111719909
TEST MAE:  13410.272976704773


# 3. Cross Validation

In [18]:
encoder = OrdinalEncoder()
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)
X_encoded = encoder.transform(X)

## 3.1 RandomizedSearchCV w/ RF

In [ ]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
param_test ={'max_depth':[None,30,50,100,150,300],
             'n_esstimators':[100,300,500]
             }
rf = RandomForestRegressor(criterion='mse', 
                          min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto',
                          max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None,
                          random_state=42, verbose=3, warm_start=False,
                          n_jobs=39,
                          ccp_alpha=0.0, max_samples=None)


rs = RandomizedSearchCV(
    estimator=rf, 
    param_distributions=param_test, 
    n_iter=10, 
    scoring='neg_mean_absolute_error',
    cv=3,
    random_state=42,
    verbose=3)

rs.fit(X_encoded, y)

In [ ]:
crossvalidation = pd.DataFrame(rs.cv_results_)
crossvalidation.to_csv('/content/drive/MyDrive/HOUSE_PRICE_PREDICTION/CV/cv_result_randomforest.csv')

## 3.2 RandomizedSearchCV w/ XGBoost


In [ ]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

fit_params={"early_stopping_rounds":50, 
            "eval_metric" : 'mae', 
            "eval_set" : [(X_train_encoded, y_train),
                          (X_val_encoded, y_val)],
            'verbose': 100}

param_test ={'max_depth':[-1,-5,10,15,30,50,100,150],
             'reg_alpha': [0, 1, 2, 5, 10, 50, 100],
             'reg_lambda': [0, 1, 5,20, 50, 100]}
xg = XGBRegressor(random_state=42,n_jobs=-1)


rs = RandomizedSearchCV(
    estimator=xg, 
    param_distributions=param_test, 
    n_iter=2,
    scoring='neg_mean_absolute_error',
    cv=3,
    random_state=1,
    verbose=3)


rs.fit(X_encoded, y, **fit_params)

In [ ]:
crossvalidation = pd.DataFrame(rs.cv_results_)
crossvalidation.to_csv('/content/drive/MyDrive/HOUSE_PRICE_PREDICTION/CV/cv_result_xgboost.csv')

## 3.3 RandomizedSearchCV w/ LGBM

In [ ]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.model_selection import RandomizedSearchCV
fit_params={"early_stopping_rounds":50, 
            "eval_metric" : 'mae', 
            "eval_set" : [(X_train_encoded, y_train),
                          (X_val_encoded, y_val)],
            'verbose': 100}

param_test ={'max_depth':[-1,-5,5,7,9,11,13,15,17,19,21,23,30,50,100,150],
             'reg_alpha': [0, 1, 2, 5, 10, 50, 100],
             'reg_lambda': [0, 1, 5,20, 50, 100],
             'subsample':[None,0.7]}
lgbm = LGBMRegressor(random_state=42,learning_rate=0.1, n_estimators=300000, n_jobs=-1)


rs = RandomizedSearchCV(
    estimator=lgbm, 
    param_distributions=param_test, 
    n_iter=100,
    scoring='neg_mean_absolute_error',
    cv=3,
    random_state=1,
    verbose=3)


rs.fit(X_train_encoded, y_train, **fit_params)

In [ ]:
crossvalidation = pd.DataFrame(rs.cv_results_)
crossvalidation.to_csv('/content/drive/MyDrive/HOUSE_PRICE_PREDICTION/CV/cv_result_lgbm.csv')

# 4. Training w/ best_params

## 4.1 RandomForest

In [61]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(criterion='mse', max_depth=50, n_estimators=300,
                          min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto',
                          max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None,
                          random_state=42, verbose=1, warm_start=False, n_jobs=-1,
                          ccp_alpha=0.0, max_samples=None)

rf.fit(X_train_encoded, y_train)

print('훈련 R^2: ', rf.score(X_train_encoded, y_train))
print('검증 R^2: ', rf.score(X_val_encoded, y_val))
print('TEST R^2: ', rf.score(X_test_encoded, y_test))

print('\n훈련 MAE: ', mean_absolute_error(rf.predict(X_train_encoded), y_train))
print('검증 MAE: ', mean_absolute_error(rf.predict(X_val_encoded), y_val))
print('TEST MAE: ', mean_absolute_error(rf.predict(X_test_encoded), y_test))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   46.4s finished
[Parallel(n_jobs=40)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done 120 tasks      | elapsed:    0.7s
[Parallel(n_jobs=40)]: Done 300 out of 300 | elapsed:    1.6s finished
[Parallel(n_jobs=40)]: Using backend ThreadingBackend with 40 concurrent workers.


훈련 R^2:  0.9961402508698601


[Parallel(n_jobs=40)]: Done 120 tasks      | elapsed:    0.2s
[Parallel(n_jobs=40)]: Done 300 out of 300 | elapsed:    0.4s finished
[Parallel(n_jobs=40)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done 120 tasks      | elapsed:    0.1s


검증 R^2:  0.9707732874824994


[Parallel(n_jobs=40)]: Done 300 out of 300 | elapsed:    0.1s finished
[Parallel(n_jobs=40)]: Using backend ThreadingBackend with 40 concurrent workers.


...

[Parallel(n_jobs=40)]: Done 120 tasks      | elapsed:    0.6s
[Parallel(n_jobs=40)]: Done 300 out of 300 | elapsed:    1.5s finished
[Parallel(n_jobs=40)]: Using backend ThreadingBackend with 40 concurrent workers.



훈련 MAE:  1785.6016348292383


[Parallel(n_jobs=40)]: Done 120 tasks      | elapsed:    0.2s


검증 MAE:  4809.630988698421
TEST MAE:  3820.3421467066905


[Parallel(n_jobs=40)]: Done 300 out of 300 | elapsed:    0.4s finished
[Parallel(n_jobs=40)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done 120 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done 300 out of 300 | elapsed:    0.1s finished


In [62]:
import pickle
filename = f'/content/drive/MyDrive/HOUSE_PRICE_PREDICTION/Model/randomforest.sav'
pickle.dump(rf, open(filename, 'wb'))

## 4.2 XGBoost

In [51]:
from xgboost import XGBRegressor
xgboost = XGBRegressor(learning_rate=0.01, max_depth=15, n_estimators=300000, 
                         num_leaves=250, reg_alpha=1, reg_lambda=1, subsample=0.7,
                         random_state=42, n_jobs=-1)

fit_params={"early_stopping_rounds":50, 
            "eval_metric" : 'mae', 
            "eval_set" : [(X_train_encoded, y_train),
                          (X_val_encoded, y_val)],
            'verbose': 100}
            
xgboost.fit(X_train_encoded, y_train,**fit_params)
print('훈련 R^2: ', xgboost.score(X_train_encoded, y_train))
print('검증 R^2: ', xgboost.score(X_val_encoded, y_val))
print('TEST R^2: ', xgboost.score(X_test_encoded, y_test))

print('\n훈련 MAE: ', mean_absolute_error(xgboost.predict(X_train_encoded), y_train))
print('검증 MAE: ', mean_absolute_error(xgboost.predict(X_val_encoded), y_val))
print('TEST MAE: ', mean_absolute_error(xgboost.predict(X_test_encoded), y_test))

[09:17:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:77137	validation_1-mae:77180.1
Multiple eval metrics have been passed: 'validation_1-mae' will be used for early stopping.

Will train until validation_1-mae hasn't improved in 50 rounds.
[100]	validation_0-mae:28753	validation_1-mae:28852.9
[200]	validation_0-mae:11366.8	validation_1-mae:12037.7
[300]	validation_0-mae:5558.2	validation_1-mae:6837.95
[400]	validation_0-mae:3674.38	validation_1-mae:5360.9
[500]	validation_0-mae:2954.27	validation_1-mae:4867.79
[600]	validation_0-mae:2595.45	validation_1-mae:4639.38
[700]	validation_0-mae:2368.31	validation_1-mae:4499.74
[800]	validation_0-mae:2215.67	validation_1-mae:4413.76
[900]	validation_0-mae:2082.68	validation_1-mae:4340
[1000]	validation_0-mae:1959.05	validation_1-mae:4275.85
[1100]	validation_0-mae:1861.48	validation_1-mae:4231.21
[1200]	validation_0-mae:1769.46	validation_1-mae:4192

In [52]:
import pickle
filename = f'/content/drive/MyDrive/HOUSE_PRICE_PREDICTION/Model/xgboost.sav'
pickle.dump(xgboost, open(filename, 'wb'))

## 4.3 LGBM

In [54]:
from lightgbm import LGBMRegressor
lightgbm = LGBMRegressor(learning_rate=0.01, max_depth=15, n_estimators=300000, 
                         num_leaves=250, reg_alpha=1, reg_lambda=1, subsample=0.7,
                         random_state=42, n_jobs=-1)

fit_params={"early_stopping_rounds":50, 
            "eval_metric" : 'mae', 
            "eval_set" : [(X_train_encoded, y_train),
                          (X_val_encoded, y_val)],
            'verbose': 100}
            
lightgbm.fit(X_train_encoded, y_train,**fit_params)
print('훈련 R^2: ', lightgbm.score(X_train_encoded, y_train))
print('검증 R^2: ', lightgbm.score(X_val_encoded, y_val))
print('TEST R^2: ', lightgbm.score(X_test_encoded, y_test))

print('\n훈련 MAE: ', mean_absolute_error(lightgbm.predict(X_train_encoded), y_train))
print('검증 MAE: ', mean_absolute_error(lightgbm.predict(X_val_encoded), y_val))
print('TEST MAE: ', mean_absolute_error(lightgbm.predict(X_test_encoded), y_test))

Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 6.83816e+08	training's l1: 18421.9	valid_1's l2: 7.0786e+08	valid_1's l1: 18592.1
[200]	training's l2: 2.7973e+08	training's l1: 11899.8	valid_1's l2: 3.04941e+08	valid_1's l1: 12179.7
[300]	training's l2: 1.77599e+08	training's l1: 9358.93	valid_1's l2: 2.05706e+08	valid_1's l1: 9727.37
[400]	training's l2: 1.3549e+08	training's l1: 8018.28	valid_1's l2: 1.65336e+08	valid_1's l1: 8429.56
[500]	training's l2: 1.14928e+08	training's l1: 7298.94	valid_1's l2: 1.46746e+08	valid_1's l1: 7747.67
[600]	training's l2: 1.00203e+08	training's l1: 6735.05	valid_1's l2: 1.33095e+08	valid_1's l1: 7213.18
[700]	training's l2: 8.94571e+07	training's l1: 6317.52	valid_1's l2: 1.23456e+08	valid_1's l1: 6827.02
[800]	training's l2: 8.11772e+07	training's l1: 5987	valid_1's l2: 1.15996e+08	valid_1's l1: 6524.82
[900]	training's l2: 7.43908e+07	training's l1: 5712.75	valid_1's l2: 1.09922e+08	valid_1's l1: 6276.69
[1000]	t

In [55]:
import pickle
filename = f'/content/drive/MyDrive/HOUSE_PRICE_PREDICTION/Model/lightgbm.sav'
pickle.dump(lightgbm, open(filename, 'wb'))